In [1]:
import requests
import sqlalchemy
import pandas as pd
import numpy as np

In [2]:
%load_ext autoreload
%autoreload 2

### Postgres Demo

In [5]:
import pandas as pd

In [6]:
from sqlalchemy import Table, Column
from sqlalchemy.dialects import postgresql as pgt

In [11]:
from pg import get_pg
pg = get_pg('estimate')

In [170]:
### Show schemas
pg.read("""SELECT schema_name FROM information_schema.schemata;""")

,schema_name
0,pg_toast
1,pg_catalog
2,public
3,information_schema
4,v1
5,v2


In [153]:
### Reflect the metadata
meta = sqlalchemy.MetaData(bind=pg.engine,schema='v1')
meta.reflect()

In [165]:
### Show tables
pd.Series([str(t) for t in meta.tables])
#pd.DataFrame({k:t.__dict__ for k,t in meta.tables.items()}).T

0     v1.raw_events
1    v1.raw_events2
dtype: object

In [224]:
pg.read("""
SELECT *
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog' AND 
    schemaname != 'information_schema';
""")

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,v1,raw_events2,suhel,None,True,False,False,False


In [174]:
### For a given table, show the columns
pd.DataFrame({c.name:c.__dict__ for c in meta.tables['v1.raw_events'].columns}).T

,key,name,table,type,is_literal,primary_key,_user_defined_nullable,nullable,default,server_default,...,constraints,foreign_keys,comment,computed,identity,_creation_order,dispatch,comparator,_memoized_keys,description
id,id,id,v1.raw_events,INTEGER,False,True,False,False,None,DefaultClause(<sqlalchemy.sql.elements.TextCla...,...,{},{},None,None,None,93,<sqlalchemy.event.base.DDLEventsDispatch objec...,<sqlalchemy.sql.sqltypes._LookupExpressionAdap...,(comparator),id
event_name,event_name,event_name,v1.raw_events,VARCHAR,False,False,False,False,None,None,...,{},{},None,None,None,94,<sqlalchemy.event.base.DDLEventsDispatch objec...,<sqlalchemy.sql.sqltypes.Concatenable.Comparat...,(comparator),event_name
inputs,inputs,inputs,v1.raw_events,JSONB,False,False,True,True,None,None,...,{},{},None,None,None,95,<sqlalchemy.event.base.DDLEventsDispatch objec...,<sqlalchemy.dialects.postgresql.json.JSONB.Com...,(comparator),inputs
outputs,outputs,outputs,v1.raw_events,JSONB,False,False,True,True,None,None,...,{},{},None,None,None,96,<sqlalchemy.event.base.DDLEventsDispatch objec...,<sqlalchemy.dialects.postgresql.json.JSONB.Com...,(comparator),outputs
metadata,metadata,metadata,v1.raw_events,JSONB,False,False,True,True,None,None,...,{},{},None,None,None,97,<sqlalchemy.event.base.DDLEventsDispatch objec...,<sqlalchemy.dialects.postgresql.json.JSONB.Com...,(comparator),metadata


In [225]:
### Define a table
rawEvents = Table('raw_events',meta,
    Column('id', pgt.INTEGER, primary_key=True, autoincrement=True),
    Column('event_name', pgt.VARCHAR, nullable=False),
    Column('inputs', pgt.JSONB),
    Column('outputs', pgt.JSONB),
    Column('metadata', pgt.JSONB),
    extend_existing=True,
)

In [230]:
### Create the table 
rawEvents.create()

In [223]:
### Drop the table
rawEvents.drop()

In [226]:
### Instantiate example data
captured_examples = """
{'event_name': 'connect', 'inputs': None, 'outputs': None}
{'event_name': 'disconnect', 'inputs': None, 'outputs': None}
{'event_name': 'start_session', 'inputs': {'userName': 'Suhel', 'sessionTag': 'debug'}, 'outputs': 'bb3f11f57b47ed89e02067c5559e1c23dffc6a9e'}
{'event_name': 'fetch_prompt', 'inputs': {}, 'outputs': {'n1': 99, 'n2': 96, 'promptModel': 'random2digits', 'accuracyModel': 'estimate'}}
{'event_name': 'fetch_prompt', 'inputs': {}, 'outputs': {'n1': 29, 'n2': 18, 'promptModel': 'random2digits', 'accuracyModel': 'estimate'}}
{'event_name': 'start_session', 'inputs': {'userName': 'Suhel', 'sessionTag': 'debug'}, 'outputs': 'bb3f11f57b47ed89e02067c5559e1c23dffc6a9e'}
{'event_name': 'fetch_prompt', 'inputs': {}, 'outputs': {'n1': 19, 'n2': 14, 'promptModel': 'random2digits', 'accuracyModel': 'estimate'}}
{'event_name': 'connect', 'inputs': None, 'outputs': None}
{'event_name': 'start_session', 'inputs': {'userName': 'Suhel', 'sessionTag': 'debug'}, 'outputs': 'bb3f11f57b47ed89e02067c5559e1c23dffc6a9e'}
{'event_name': 'fetch_prompt', 'inputs': {}, 'outputs': {'n1': 33, 'n2': 12, 'promptModel': 'random2digits', 'accuracyModel': 'estimate'}}
{'event_name': 'send_answer', 'inputs': {'answer': '234234', 'elapsedTime': 2, 'prompt': {'n1': 33, 'n2': 12}}, 'outputs': {'providedAnswer': 234234.0, 'correctAnswer': 396.0, 'pctError': 590.5, 'score': 0.08453085376162299}}
{'event_name': 'fetch_prompt', 'inputs': {}, 'outputs': {'n1': 84, 'n2': 71, 'promptModel': 'random2digits', 'accuracyModel': 'estimate'}}
{'event_name': 'send_answer', 'inputs': {'answer': '234', 'elapsedTime': 9, 'prompt': {'n1': 84, 'n2': 71}}, 'outputs': {'providedAnswer': 234.0, 'correctAnswer': 5964.0, 'pctError': 0.9607645875251509, 'score': 1.9617706237424548}}
{'event_name': 'fetch_prompt', 'inputs': {}, 'outputs': {'n1': 88, 'n2': 90, 'promptModel': 'random2digits', 'accuracyModel': 'estimate'}}
{'event_name': 'disconnect', 'inputs': None, 'outputs': None}
"""
event_log = pd.DataFrame(list(map(eval, captured_examples[1:-1].split('\n'))))
event_log

,event_name,inputs,outputs
0,connect,None,None
1,disconnect,None,None
2,start_session,"{'userName': 'Suhel', 'sessionTag': 'debug'}",bb3f11f57b47ed89e02067c5559e1c23dffc6a9e
3,fetch_prompt,{},"{'n1': 99, 'n2': 96, 'promptModel': 'random2di..."
4,fetch_prompt,{},"{'n1': 29, 'n2': 18, 'promptModel': 'random2di..."
5,start_session,"{'userName': 'Suhel', 'sessionTag': 'debug'}",bb3f11f57b47ed89e02067c5559e1c23dffc6a9e
6,fetch_prompt,{},"{'n1': 19, 'n2': 14, 'promptModel': 'random2di..."
7,connect,None,None
8,start_session,"{'userName': 'Suhel', 'sessionTag': 'debug'}",bb3f11f57b47ed89e02067c5559e1c23dffc6a9e
9,fetch_prompt,{},"{'n1': 33, 'n2': 12, 'promptModel': 'random2di..."


In [237]:
### Insert a single row (can run this multiple times)
row = event_log.sample().iloc[0]

pg.engine.execute(
    rawEvents.insert().values(**row)
)

In [238]:
### Read rows directly
pg.read("""SELECT * FROM v1.raw_events""")

,id,event_name,inputs,outputs,metadata
0,1,disconnect,None,None,None
1,2,start_session,"{'userName': 'Suhel', 'sessionTag': 'debug'}",bb3f11f57b47ed89e02067c5559e1c23dffc6a9e,None
2,3,fetch_prompt,{},"{'n1': 88, 'n2': 90, 'promptModel': 'random2di...",None


In [284]:
### Read rows through the cursor (ugh)
pg.engine.execute( 
    rawEvents.select().limit(10)
).fetchall()

[(19, 'connect', None, None, None),
 (20, 'disconnect', None, None, None),
 (21, 'start_session', {'userName': 'Suhel', 'sessionTag': 'debug'}, 'bb3f11f57b47ed89e02067c5559e1c23dffc6a9e', None),
 (22, 'fetch_prompt', {}, {'n1': 99, 'n2': 96, 'promptModel': 'random2digits', 'accuracyModel': 'estimate'}, None),
 (23, 'fetch_prompt', {}, {'n1': 29, 'n2': 18, 'promptModel': 'random2digits', 'accuracyModel': 'estimate'}, None),
 (24, 'start_session', {'userName': 'Suhel', 'sessionTag': 'debug'}, 'bb3f11f57b47ed89e02067c5559e1c23dffc6a9e', None),
 (25, 'fetch_prompt', {}, {'n1': 19, 'n2': 14, 'promptModel': 'random2digits', 'accuracyModel': 'estimate'}, None),
 (26, 'connect', None, None, None),
 (27, 'start_session', {'userName': 'Suhel', 'sessionTag': 'debug'}, 'bb3f11f57b47ed89e02067c5559e1c23dffc6a9e', None),
 (28, 'fetch_prompt', {}, {'n1': 33, 'n2': 12, 'promptModel': 'random2digits', 'accuracyModel': 'estimate'}, None)]

In [288]:
### lol or do it this way

print(
    rawEvents.select().limit(10).compile(compile_kwargs={"literal_binds": True})
)
pg.read(str(
    rawEvents.select().limit(10).compile(compile_kwargs={"literal_binds": True})
))

SELECT v1.raw_events.id, v1.raw_events.event_name, v1.raw_events.inputs, v1.raw_events.outputs, v1.raw_events.metadata 
FROM v1.raw_events 
 LIMIT 10


,id,event_name,inputs,outputs,metadata
0,19,connect,None,None,None
1,20,disconnect,None,None,None
2,21,start_session,"{'userName': 'Suhel', 'sessionTag': 'debug'}",bb3f11f57b47ed89e02067c5559e1c23dffc6a9e,None
3,22,fetch_prompt,{},"{'n1': 99, 'n2': 96, 'promptModel': 'random2di...",None
4,23,fetch_prompt,{},"{'n1': 29, 'n2': 18, 'promptModel': 'random2di...",None
5,24,start_session,"{'userName': 'Suhel', 'sessionTag': 'debug'}",bb3f11f57b47ed89e02067c5559e1c23dffc6a9e,None
6,25,fetch_prompt,{},"{'n1': 19, 'n2': 14, 'promptModel': 'random2di...",None
7,26,connect,None,None,None
8,27,start_session,"{'userName': 'Suhel', 'sessionTag': 'debug'}",bb3f11f57b47ed89e02067c5559e1c23dffc6a9e,None
9,28,fetch_prompt,{},"{'n1': 33, 'n2': 12, 'promptModel': 'random2di...",None


In [263]:
### Insert the whole df
dtypes = {c.name:c.type for c in rawEvents.columns}  # you need to get the dtypes first (wtf)
event_log.to_sql(name='raw_events',schema='v1',index=None,con=pg.engine, if_exists='append', dtype=dtypes)
pg.read("""SELECT * FROM v1.raw_events""")[:10]

,id,event_name,inputs,outputs,metadata
0,19,connect,None,None,None
1,20,disconnect,None,None,None
2,21,start_session,"{'userName': 'Suhel', 'sessionTag': 'debug'}",bb3f11f57b47ed89e02067c5559e1c23dffc6a9e,None
3,22,fetch_prompt,{},"{'n1': 99, 'n2': 96, 'promptModel': 'random2di...",None
4,23,fetch_prompt,{},"{'n1': 29, 'n2': 18, 'promptModel': 'random2di...",None
5,24,start_session,"{'userName': 'Suhel', 'sessionTag': 'debug'}",bb3f11f57b47ed89e02067c5559e1c23dffc6a9e,None
6,25,fetch_prompt,{},"{'n1': 19, 'n2': 14, 'promptModel': 'random2di...",None
7,26,connect,None,None,None
8,27,start_session,"{'userName': 'Suhel', 'sessionTag': 'debug'}",bb3f11f57b47ed89e02067c5559e1c23dffc6a9e,None
9,28,fetch_prompt,{},"{'n1': 33, 'n2': 12, 'promptModel': 'random2di...",None


In [252]:
### Truncate everything
pg.engine.execute(rawEvents.delete())

### Init DB

### Testing some shiz

In [6]:
from db.database import db_session
from db.models import (Session, Interactions, RawEventLog)

##### Sessions

In [4]:
sess = Session(socket_id='asdlkfjs',session_tag='practice',user='suhel')

In [5]:
db_session.add(sess)
db_session.commit()

##### Raw Event Logs

In [4]:
### Instantiate example data
captured_examples = """
{'event_name': 'connect', 'inputs': None, 'outputs': None}
{'event_name': 'disconnect', 'inputs': None, 'outputs': None}
{'event_name': 'start_session', 'inputs': {'userName': 'Suhel', 'sessionTag': 'debug'}, 'outputs': 'bb3f11f57b47ed89e02067c5559e1c23dffc6a9e'}
{'event_name': 'fetch_prompt', 'inputs': {}, 'outputs': {'n1': 99, 'n2': 96, 'promptModel': 'random2digits', 'accuracyModel': 'estimate'}}
{'event_name': 'fetch_prompt', 'inputs': {}, 'outputs': {'n1': 29, 'n2': 18, 'promptModel': 'random2digits', 'accuracyModel': 'estimate'}}
{'event_name': 'start_session', 'inputs': {'userName': 'Suhel', 'sessionTag': 'debug'}, 'outputs': 'bb3f11f57b47ed89e02067c5559e1c23dffc6a9e'}
{'event_name': 'fetch_prompt', 'inputs': {}, 'outputs': {'n1': 19, 'n2': 14, 'promptModel': 'random2digits', 'accuracyModel': 'estimate'}}
{'event_name': 'connect', 'inputs': None, 'outputs': None}
{'event_name': 'start_session', 'inputs': {'userName': 'Suhel', 'sessionTag': 'debug'}, 'outputs': 'bb3f11f57b47ed89e02067c5559e1c23dffc6a9e'}
{'event_name': 'fetch_prompt', 'inputs': {}, 'outputs': {'n1': 33, 'n2': 12, 'promptModel': 'random2digits', 'accuracyModel': 'estimate'}}
{'event_name': 'send_answer', 'inputs': {'answer': '234234', 'elapsedTime': 2, 'prompt': {'n1': 33, 'n2': 12}}, 'outputs': {'providedAnswer': 234234.0, 'correctAnswer': 396.0, 'pctError': 590.5, 'score': 0.08453085376162299}}
{'event_name': 'fetch_prompt', 'inputs': {}, 'outputs': {'n1': 84, 'n2': 71, 'promptModel': 'random2digits', 'accuracyModel': 'estimate'}}
{'event_name': 'send_answer', 'inputs': {'answer': '234', 'elapsedTime': 9, 'prompt': {'n1': 84, 'n2': 71}}, 'outputs': {'providedAnswer': 234.0, 'correctAnswer': 5964.0, 'pctError': 0.9607645875251509, 'score': 1.9617706237424548}}
{'event_name': 'fetch_prompt', 'inputs': {}, 'outputs': {'n1': 88, 'n2': 90, 'promptModel': 'random2digits', 'accuracyModel': 'estimate'}}
{'event_name': 'disconnect', 'inputs': None, 'outputs': None}
"""
event_log = pd.DataFrame(list(map(eval, captured_examples[1:-1].split('\n'))))
event_log.at[:,'meta'] = [{'timestamp':pd.Timestamp.now().isoformat(), 'socket_id':'FUCK'}]*event_log.shape[0]

In [ ]:
for index,row in event_log.iterrows():
    re = RawEventLog(**row)
    db_session.add(re)

In [9]:
db_session.commit()

In [10]:
res=RawEventLog.query.all()

In [11]:
res[3].__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x7f50d2deaee0>,
 'event_name': 'fetch_prompt',
 'outputs': {'n1': 99,
  'n2': 96,
  'promptModel': 'random2digits',
  'accuracyModel': 'estimate'},
 'meta': {'socket_id': 'FUCK', 'timestamp': '2022-02-09T04:59:06.443718'},
 'inputs': {},
 'event_id': 4}

##### Interactions

In [5]:
row = event_log.loc[10].copy()

In [6]:
row

event_name                                          send_answer
inputs        {'answer': '234234', 'elapsedTime': 2, 'prompt...
outputs       {'providedAnswer': 234234.0, 'correctAnswer': ...
meta          {'timestamp': '2022-02-09T05:07:23.564978', 's...
Name: 10, dtype: object

In [7]:
prompt = row['inputs'].pop('prompt')
response = row['inputs']

In [8]:
review = {}

In [9]:
session_id = 'sdflkjsdf;ljslfs;dfl'

In [10]:
i = Interactions(session_id,prompt,response,review)

In [11]:
db_session.add(i)

### Explore the db yo

In [2]:
from pg import get_pg
pg = get_pg('estimate_t1')

In [4]:
pg.read("""
SELECT *
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog' AND 
    schemaname != 'information_schema';
""")

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,public,raw_events,suhel,None,True,False,False,False
1,public,sessions,suhel,None,True,False,True,False
2,public,interactions,suhel,None,True,False,True,False


In [11]:
pg.read("""
SELECT *
FROM raw_events
""").tail(10)

,event_id,event_name,inputs,outputs,meta
30,31,send_answer,"{'answer': '234', 'prompt': {'n1': 31, 'n2': 5...","{'score': 6.507230255839821, 'pctError': 0.869...","{'socket_id': 'wH0iu0Dp17Pk0hdPAAAB', 'timesta..."
31,32,fetch_prompt,{'sessionId': 'a342b5919bff18af0e8e43ed50f4074...,"{'n1': 70, 'n2': 66, 'promptModel': 'random2di...","{'socket_id': 'wH0iu0Dp17Pk0hdPAAAB', 'timesta..."
32,33,send_answer,"{'answer': '23524', 'prompt': {'n1': 70, 'n2':...","{'score': 9.819758544465227, 'pctError': 4.091...","{'socket_id': 'wH0iu0Dp17Pk0hdPAAAB', 'timesta..."
33,34,fetch_prompt,{'sessionId': 'a342b5919bff18af0e8e43ed50f4074...,"{'n1': 68, 'n2': 38, 'promptModel': 'random2di...","{'socket_id': 'wH0iu0Dp17Pk0hdPAAAB', 'timesta..."
34,35,send_answer,"{'answer': '2356', 'prompt': {'n1': 68, 'n2': ...","{'score': 45.588235294117645, 'pctError': 0.08...","{'socket_id': 'wH0iu0Dp17Pk0hdPAAAB', 'timesta..."
35,36,fetch_prompt,{'sessionId': 'a342b5919bff18af0e8e43ed50f4074...,"{'n1': 38, 'n2': 11, 'promptModel': 'random2di...","{'socket_id': 'wH0iu0Dp17Pk0hdPAAAB', 'timesta..."
36,37,send_answer,"{'answer': '2357', 'prompt': {'n1': 38, 'n2': ...","{'score': 8.867204072974118, 'pctError': 4.638...","{'socket_id': 'wH0iu0Dp17Pk0hdPAAAB', 'timesta..."
37,38,fetch_prompt,{'sessionId': 'a342b5919bff18af0e8e43ed50f4074...,"{'n1': 88, 'n2': 30, 'promptModel': 'random2di...","{'socket_id': 'wH0iu0Dp17Pk0hdPAAAB', 'timesta..."
38,39,send_answer,"{'answer': '1', 'prompt': {'n1': 88, 'n2': 30}...","{'score': 0.01893939393939394, 'pctError': 0.9...","{'socket_id': 'wH0iu0Dp17Pk0hdPAAAB', 'timesta..."
39,40,fetch_prompt,{'sessionId': 'a342b5919bff18af0e8e43ed50f4074...,"{'n1': 69, 'n2': 10, 'promptModel': 'random2di...","{'socket_id': 'wH0iu0Dp17Pk0hdPAAAB', 'timesta..."


In [12]:
pg.read("""
SELECT *
FROM sessions
""").tail(10)

,session_id,socket_id,session_tag,user,timestamp
0,9d9097a2aeb067e4c325e91026fc8d016c199066,wH0iu0Dp17Pk0hdPAAAB,test,Suhel,2022-02-09 05:32:21.125715
1,a342b5919bff18af0e8e43ed50f40745f4daad9f,wH0iu0Dp17Pk0hdPAAAB,debug,Suhel,2022-02-09 05:32:40.787771


In [13]:
pg.read("""
SELECT *
FROM interactions
""").tail(10)

,interaction_id,session_id,prompt,response,review,timestamp_received
7,8,a342b5919bff18af0e8e43ed50f40745f4daad9f,"{'n1': 58, 'n2': 64}","{'answer': '2342', 'elapsedTime': 1}","{'score': 31.54633620689655, 'pctError': 0.369...",2022-02-09 05:32:42.114777
8,9,a342b5919bff18af0e8e43ed50f40745f4daad9f,"{'n1': 13, 'n2': 12}","{'answer': '2342', 'elapsedTime': 0}","{'score': 3.330486763450043, 'pctError': 14.01...",2022-02-09 05:32:43.259017
9,10,a342b5919bff18af0e8e43ed50f40745f4daad9f,"{'n1': 72, 'n2': 11}","{'answer': '1355', 'elapsedTime': 1}","{'score': 29.225092250922508, 'pctError': 0.71...",2022-02-09 05:32:44.855707
10,11,a342b5919bff18af0e8e43ed50f40745f4daad9f,"{'n1': 40, 'n2': 80}","{'answer': '34574', 'elapsedTime': 0}","{'score': 4.6277549603748485, 'pctError': 9.80...",2022-02-09 05:32:46.140025
11,12,a342b5919bff18af0e8e43ed50f40745f4daad9f,"{'n1': 12, 'n2': 51}","{'answer': '43564', 'elapsedTime': 0}","{'score': 0.7024148379395831, 'pctError': 70.1...",2022-02-09 05:32:47.503673
12,13,a342b5919bff18af0e8e43ed50f40745f4daad9f,"{'n1': 31, 'n2': 58}","{'answer': '234', 'elapsedTime': 0}","{'score': 6.507230255839821, 'pctError': 0.869...",2022-02-09 05:32:49.389717
13,14,a342b5919bff18af0e8e43ed50f40745f4daad9f,"{'n1': 70, 'n2': 66}","{'answer': '23524', 'elapsedTime': 1}","{'score': 9.819758544465227, 'pctError': 4.091...",2022-02-09 05:32:51.736656
14,15,a342b5919bff18af0e8e43ed50f40745f4daad9f,"{'n1': 68, 'n2': 38}","{'answer': '2356', 'elapsedTime': 0}","{'score': 45.588235294117645, 'pctError': 0.08...",2022-02-09 05:32:53.213654
15,16,a342b5919bff18af0e8e43ed50f40745f4daad9f,"{'n1': 38, 'n2': 11}","{'answer': '2357', 'elapsedTime': 1}","{'score': 8.867204072974118, 'pctError': 4.638...",2022-02-09 05:32:54.628937
16,17,a342b5919bff18af0e8e43ed50f40745f4daad9f,"{'n1': 88, 'n2': 30}","{'answer': '1', 'elapsedTime': 3}","{'score': 0.01893939393939394, 'pctError': 0.9...",2022-02-09 05:32:58.068604
